In [ ]:
!pip install -q nnAudio -qq
import torch
from nnAudio.Spectrogram import CQT1992v2

In [ ]:
!pip install -U efficientnet

In [ ]:
import efficientnet.keras as efn

In [ ]:
import os
import json
import random
import collections
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import math
from random import shuffle

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.utils import Sequence
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from tensorflow.keras.optimizers import Adam


from tensorflow.keras import models, layers
from tensorflow.keras.layers import Dense, Dropout, Activation, Input, BatchNormalization, GlobalAveragePooling2D


from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn import model_selection as sk_model_selection

In [ ]:
train = pd.read_csv('../input/g2net-gravitational-wave-detection/training_labels.csv')
datosValidacion = pd.read_csv('../input/g2net-gravitational-wave-detection/sample_submission.csv')
test = datosValidacion

display(train.head(3))
display(test.head(3))

In [ ]:
def obtenerRutaDeImagenEntrenamiento(IDImagen):
    return "../input/g2net-gravitational-wave-detection/train/{}/{}/{}/{}.npy".format(
        IDImagen[0], IDImagen[1], IDImagen[2], IDImagen)

def obtenerRutaDeImagenDePrueba(IDImagen):
    return "../input/g2net-gravitational-wave-detection/test/{}/{}/{}/{}.npy".format(
        IDImagen[0], IDImagen[1], IDImagen[2], IDImagen)

train['file_path'] = train['id'].apply(obtenerRutaDeImagenEntrenamiento)
test['file_path'] = test['id'].apply(obtenerRutaDeImagenDePrueba)

display(train.head(3))
display(test.head(3))

In [ ]:
x_train, x_val = sk_model_selection.train_test_split(
    train, 
    test_size=0.05, 
    random_state=42
)
print(len(x_train))
print(len(x_val))

In [ ]:
class Dataset(Sequence):
    def __init__(self,df,esEntrenamiento=True,tamanoLote=32,shuffle=True):
        self.id = df["id"].values
        self.ruta = df["file_path"].values
        self.y =  df["target"].values
        self.esEntrenamiento = esEntrenamiento
        self.tamanoLote = tamanoLote
        self.shuffle = shuffle
        self.transformadaDeOnda = CQT1992v2(sr=2048, fmin=20, fmax=1024, hop_length=64)

    def __len__(self):
        return math.ceil(len(self.id)/self.tamanoLote)
    
    def aplicarTransformadaQ(self,pathx,transform): 
        ondas = np.load(pathx)
        ondas = np.hstack(ondas)
        ondas = ondas / np.max(ondas)
        ondas = torch.from_numpy(ondas).float()
        imagen = transform(ondas)
        imagen = np.array(imagen)
        imagen = np.transpose(imagen,(1,2,0))
        return imagen 
    
    def __getitem__(self,ids):
        rutaDeLotes = self.ruta[ids * self.tamanoLote:(ids + 1) * self.tamanoLote]
        
        if self.y is not None:
            loteY = self.y[ids * self.tamanoLote: (ids + 1) * self.tamanoLote]
            
        listaX = np.array([self.aplicarTransformadaQ(x,self.transformadaDeOnda) for x in rutaDeLotes])
        loteX = np.stack(listaX)
        if self.esEntrenamiento:
            return loteX, loteY
        else:
            return loteX

In [ ]:
DatasetDeEntrenamiento = Dataset(x_train) 
DatasetDeValidacion = Dataset(x_val) 

In [ ]:
def crearModelo(): 
    inputs = layers.Input(shape=(69,193,1))
    capasEfficientnet = efn.EfficientNetB7(include_top=False,input_shape=(),weights='imagenet',pooling='avg')
    modelo = Sequential()
    
    modelo.add(inputs)
    modelo.add(keras.layers.Conv2D(3,3,activation='relu',padding='same'))
    modelo.add(capasEfficientnet)

    modelo.add(Dense(1, activation="sigmoid"))
    
    modelo.compile(optimizer = Adam(lr = 0.00005),
                loss = "binary_crossentropy",
                metrics = ["acc"])
    return modelo

modelo = crearModelo()
modelo.summary()

In [ ]:
unaPrueba = Dataset(x_train[:1000])

In [ ]:

train_history = modelo.fit(
    unaPrueba,
    epochs = 1,
    validation_data = DatasetDeValidacion
)

In [ ]:
DatasetAPredecir = Dataset(test,esEntrenamiento=False)

In [ ]:
predicciones = modelo.predict(DatasetAPredecir)
predicciones = predicciones.reshape(-1)

In [ ]:
submission = pd.DataFrame({'id':datosValidacion['id'],'target':predicciones})


In [ ]:
submission

In [ ]:
submission.to_csv('submission.csv',index=False)